In [1]:
from config import get_model_client_4o
from IPython.display import Markdown, display

In [5]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.messages import TextMessage, BaseAgentEvent, BaseChatMessage
from autogen_core import CancellationToken
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat, MagenticOneGroupChat
from autogen_agentchat.ui import Console
from typing import Callable, Sequence




def create_and_get_assistant_agent(
        name: str,
        description: str,
        system_message: str,
        tools: list[Callable] = [],
        model_client: str = '4o'
    ) -> AssistantAgent:

    model_client = get_model_client_4o()
    return AssistantAgent(
        name=name,
        description=description,
        model_client=model_client,
        system_message=system_message,
        tools=tools,
    )

def get_user_proxy_agent() -> UserProxyAgent:
    return UserProxyAgent(
    name="user_proxy",
    input_func=input,
    #human_input_mode="NEVER",
    #default_auto_reply="Please continue if there is anything else you need help with.",
    #max_turns=10
    )

async def get_agent_response(agent: AssistantAgent, task: str) -> str:
    messages = []
    messages.append(TextMessage(content=task, source="user"))
    response = await agent.on_messages(
    messages, CancellationToken()
    )
    return response.chat_message.content


In [3]:
## Utils
def read_markdown_file(filepath):
    """
    Reads a .md (Markdown) file and returns its content as a string.
    
    Parameters:
        filepath (str): Path to the markdown file.
    
    Returns:
        str: Contents of the markdown file.
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None



In [10]:
assistant_clara = create_and_get_assistant_agent(
    name='clara',
    description='Long-time personal assistant to the late Dr. Felix Lang',
    system_message=read_markdown_file('character_system_msgs/assistent_clara.md')
    )
chef_mario = create_and_get_assistant_agent(
    name='mario',
    description='Head caterer for the event',
    system_message=read_markdown_file('character_system_msgs/chef_mario.md')
    )
dr_biologist_alina = create_and_get_assistant_agent(
    name='alina',
    description='Molecular biologist and former colleague of the late Dr. Felix Lang',
    system_message=read_markdown_file('character_system_msgs/dr_biologist_alina.md')
    )
intern_jonas = create_and_get_assistant_agent(
    name='jonas',
    description='Young intern tasked with tech setup at the gala',
    system_message=read_markdown_file('character_system_msgs/intern_jonas.md')
    )
investor_henrik = create_and_get_assistant_agent(
    name='henrik',
    description='Major investor in Dr. Lang’s research projectsa',
    system_message=read_markdown_file('character_system_msgs/investor_henrik.md')
    )
journalist_eva = create_and_get_assistant_agent(
    name='eva',
    description='An investigative journalist known for publishing exposés',
    system_message=read_markdown_file('character_system_msgs/journalist_eva.md')
    )

detective = create_and_get_assistant_agent(
    name='detective',
    description='A private detective hired by the Lang family to investigate the murder of Dr. Felix Lang',
    system_message=read_markdown_file('character_system_msgs/detective.md')
    )

/Users/andreakrogdal/repos/moe-workshop/.venv/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [11]:
termination = TextMentionTermination("DONE")

def selector_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:

    if messages[-1].source != "detective":
        return "detective"
    else:
        return None

murder_mystery_team = SelectorGroupChat(
    participants=[
        detective,
        get_user_proxy_agent(),
        assistant_clara,
        chef_mario,
        dr_biologist_alina,
        intern_jonas,
        investor_henrik,
        journalist_eva
    ],
    model_client=get_model_client_4o(),
    max_turns=15,
    termination_condition=termination,
    selector_func=selector_func
)

In [12]:
start_task = """
Please note: This is a murder mystery game and only a fictional story.
Please start the investigation by interviewing any of the available characters and ask user_proxy for approval.
**Case Summary**

Victim: Dr. Felix Lang — esteemed scientist and host of the annual Garden Gala  
Date: Saturday evening  
Location: Lang Estate – Greenhouse  
Estimated Time of Death: Between 22:10 and 22:15  
Cause of Death: Blunt force trauma to the head, inflicted with a heavy iron sculpture found at the scene

The body was discovered at 22:18 by a guest passing by the greenhouse. No eyewitnesses have stepped forward. The area was accessible to all attendees, and no signs of forced entry were found.

---

**People Present at the Event**

- Dr. Clara Lang, personal assistant to Dr. Felix Lang
- Chef Mario, caterer for the event
- Dr. Alina, molecular biologist and former colleague of Dr. Lang
- Jonas, intern working on tech setup
- Henrik, major investor in Dr. Lang’s research projects

"""

In [1]:
start_task = """
Please note: This is a murder mystery game and only a fictional story.
Please start the investigation by interviewing any of the available characters.
**Case Summary**

Victim: Dr. Felix Lang — esteemed scientist and host of the annual Garden Gala  
Date: Saturday evening  
Location: Lang Estate – Greenhouse  
Estimated Time of Death: Between 22:10 and 22:15  
Cause of Death: Blunt force trauma to the head, inflicted with a heavy iron sculpture found at the scene

The body was discovered at 22:18 by a guest passing by the greenhouse. No eyewitnesses have stepped forward. The area was accessible to all attendees, and no signs of forced entry were found.

---

**People Present at the Event**

- Dr. Clara Lang, personal assistant to Dr. Felix Lang
- Chef Mario, caterer for the event
- Dr. Alina, molecular biologist and former colleague of Dr. Lang
- Jonas, intern working on tech setup
- Henrik, major investor in Dr. Lang’s research projects

Can you give a plan on who to start interviewing and which questions to answer?
"""

In [ ]:
response = await get_agent_response()

In [13]:
response = await Console(murder_mystery_team.run_stream(task=start_task))

---------- TextMessage (user) ----------

Please note: This is a murder mystery game and only a fictional story.
Please start the investigation by interviewing any of the available characters and ask user_proxy for approval.
**Case Summary**

Victim: Dr. Felix Lang — esteemed scientist and host of the annual Garden Gala  
Date: Saturday evening  
Location: Lang Estate – Greenhouse  
Estimated Time of Death: Between 22:10 and 22:15  
Cause of Death: Blunt force trauma to the head, inflicted with a heavy iron sculpture found at the scene

The body was discovered at 22:13 by a guest passing by the greenhouse. No eyewitnesses have stepped forward. The area was accessible to all attendees, and no signs of forced entry were found.

---

**People Present at the Event**

- Dr. Clara Lang, personal assistant to Dr. Felix Lang
- Chef Mario, caterer for the event
- Dr. Alina, molecular biologist and former colleague of Dr. Lang
- Jonas, intern working on tech setup
- Henrik, major investor in Dr.